### Guideline
\# TODO : ~~

목표에 맞게 코드를 작성하면 됩니다!

교재의 코드 참고해도 좋아요~

KoNLPy의 Okt를 사용하기 때문에 KoNLPy가 설치되어있어야 합니다!

이 노트북에서는 <b>'과제명' 피처만 사용하여 학습</b>시킵니다! 다른 피처도 사용하면 더 성능이 좋아질 가능성이 높겠죠? 나중에 시도해보시는 것 추천드려요b

# **1. 데이터 및 라이브러리 불러오기**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json
import os
import tqdm

from konlpy.tag import Okt

import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score,f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# TODO : 데이터 import (train, test, 자신의 데이터셋 위치 고려하여 작성!)


sample_submission = pd.read_csv('sample_submission.csv')

# **2. 데이터 EDA**

In [ ]:
train.head(2)

In [ ]:
test.head(2)

In [ ]:
sample_submission.head(6)

In [ ]:
# 데이터 구조 파악
print(train.shape)
print(test.shape)
print(sample_submission.shape)

### 시각화
#### 레이블 분포

In [ ]:
# 레이블 분포 확인
label_counts = train['label'].value_counts()
print(label_counts)

# 레이블 분포 시각화
plt.figure(figsize=(12, 6))
label_counts.plot(kind='bar', color='skyblue')
plt.title('Label Distribution')
plt.xlabel('Labels')
plt.ylabel('Count')
plt.show()

80% 이상을 0 레이블이 차지합니다.

불균형한 레이블 분포이고, 데이터 건수가 너무 많기 때문에 언더 샘플링, SMOTE 오버샘플링을 수행할 예정입니다.

</br>

#### 텍스트 길이

텍스트분석에서 <b>텍스트의 길이 분석이 도움되는 점</b>
1. 전처리 결정
- 이상치 제거
- 데이터 불균형 탐지 및 처리
2. 모델 선택 및 설정
- 텍스트 길이 따른 적절한 모델 선택 <br>
긴 텍스트를 처리해야 하는 경우, 단순한 모델보다 LSTM이나 Transformer와 같은 더 복잡한 모델이 필요할 수 있다.
3. 패딩 전략 수립 <br>
시퀀스 모델에서 입력 길이를 맞추기 위해 패딩을 사용할 때, 적절한 최대 시퀀스 길이를 설정하는 데 도움이 된다.<br><br>
등등...
<br>
<b>하지만 이 노트북에서는 단순한 전처리만 할 예정입니다. 참고만 해주세요!</b>

In [ ]:
length=train['과제명'].astype(str).apply(len)
plt.hist(length, bins=50, alpha=0.5, color='r', label='word') # 히스토그램 그리기 (bins = 구간 개수, label = 해당 그래프 데이터 레이블)
plt.title('histogram of length of task_name')
plt.figure(figsize=(12, 5))
plt.boxplot(length, labels=['counts'], showmeans=True)
print('과제명 길이 최댓값: {}'.format(np.max(length)))
print('과제명 길이 최솟값: {}'.format(np.min(length)))
print('과제명 길이 평균값: {}'.format(np.mean(length)))
print('과제명 길이 중간값: {}'.format(np.median(length)))

# **3. 데이터 전처리**

'과제명'만 학습에 사용합니다.

In [ ]:
X_train = train[['과제명']]
y_train = train[['label']]
test = test[['과제명']]

In [ ]:
X_train.head(2)

In [ ]:
y_train.head(2)

In [ ]:
test.head(2)

In [ ]:
# 전처리 함수 정의
def preprocessing(text, okt, remove_stopwords=False, stop_words=[]):
    # TODO : 한글 및 공백만 남기고 제거하기 (힌트: OO.OOO(r'한글 및 공백 제외 문자', '', OOOO) / 교재 521p 참고)
    text = 
    # TODO : okt로 형태소 분석 (0000, stem = True)
    word_text = 
    # stop word 제거
    if remove_stopwords:
        word_review=[token for token in word_text if not token in stop_words]
    return word_review

# stop word(불용어) 리스트
stop_words=['은','는','이','가', '하','아','것','들','의','있','되','수','보','주','등','한']

# TODO : Okt 객체 생성
okt = 

In [ ]:
# 훈련 데이터 및 테스트 데이터 전처리
clean_train_text = []
clean_test_text = []

# 데이터 건수가 많아 오래걸립니다.
for text in tqdm.tqdm(X_train['과제명']):
    try:
        clean_train_text.append(preprocessing(text, okt, remove_stopwords=True, stop_words=stop_words))
    except:
        clean_train_text.append([])

for text in tqdm.tqdm(test['과제명']):
    if isinstance(text, str):
        clean_test_text.append(preprocessing(text, okt, remove_stopwords=True, stop_words=stop_words))
    else:
        clean_test_text.append([])

In [ ]:
len(clean_train_text)

In [ ]:
len(clean_test_text)

In [ ]:
# TODO: 피처 벡터화

# 1. CountVectorizer, TfidfVectorizer 모듈 임포트


# 2. CountVectorizer 객체 생성 / 이름: cnt_vect, (파라미터: tokenizer = lamda x : x, lowercase = False)
# tokenizer 인자에는 list를 받아서 그대로 내보내는 함수를 넣어줍니다. 또한 소문자화를 하지 않도록 설정해야 에러가 나지 않습니다.

# 학습 데이터 피처 벡터화 / 이름: cnt_train_features

# 테스트 데이터 피처 벡터화 / 이름: cnt_tst_features (fit_transform / fit / transform 중 뭐가 맞을까요?)


# 3. TfidfVectorizer 객체 생성 / 이름: tfidf_vect, (파라미터: tokenizer = lamda x : x, lowercase = False)

# CountVectorizer처럼 학습, 테스트 데이터 피처 벡터화 / 이름: tfidf_train_features, tfidf_test_features



시간 상의 문제로 TF-IDF 벡터화한 데이터셋으로 계속 진행합니다.

In [ ]:
# 언더 샘플링 및 SMOTE 오버 샘플링
# *imbalanced-learn 파이썬 패키지가 설치되어 있어야 합니다. (분류 챕터 숙제하셨다면 설치완료!)
# 설치: 아나콘다 프롬프트를 관리자 권한 실행 -> conda install -c conda-forge imbalanced-learn 
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

# 언더 샘플링
under_sampler = RandomUnderSampler(
        sampling_strategy={0: 5000}, # 0 레이블을 5000개로 감소
        random_state=42
        )
X_under, y_under = under_sampler.fit_resample(train_features_tfidf, y_train)

# SMOTE 오버 샘플링
smote = SMOTE(
        sampling_strategy='not majority',  # 다수 클래스 제외한 나머지만 증가
        random_state=42,
        k_neighbors=5
        )
X_balanced, y_balanced = smote.fit_resample(X_under, y_under) 

In [ ]:
# 최종 학습 피처 데이터 세트
X_balanced

In [ ]:
# 최종 학습 레이블 데이터 세트
y_balanced.label.value_counts(sort=False)

# **4. 모델링**

In [ ]:
# 훈련 데이터 셋과 검증 데이터 셋으로 분리
TEST_SIZE=0.2
RANDOM_SEED=42

# TODO: 빈칸 채우기!
train_x, eval_x, train_y, eval_y=_____________(__________, __________, __________, __________)

In [ ]:
# TODO: 랜덤포레스트로 모델링
# 랜덤포레스트 모듈 임포트

# RandomForestClassifier 모델 생성 (결정 트리 개수 100개, verbose = 1, n_jobs = -1)

# 학습 데이터로 모델 훈련


In [ ]:
# TODO: 모델 검증 (빈칸 채우기)
forest.score(_______, _______)

# **5. 예측 및 제출**

In [ ]:
# TODO: 피처 벡터화한 테스트 데이터 사용해 랜덤포레스트 모델 예측, test_predictions에 저장


In [ ]:
# TODO: 샘플 제출용 데이터프레임에 예측 결과 업데이트 (빈칸 채우기)
sample_submission['label'] = ________________

In [ ]:
sample_submission.head(10)

In [ ]:
# 제출할 예측 결과 파일 생성!
sample_submission.to_csv('submission0.csv', index=False)

직접 사이트에 제출해보셔도 됩니다!

점수는.. 기대 안하시는게..